## Loading up GDrive

In [ ]:
from google.colab import drive
drive.mount(
    "/content/drive",
    force_remount = True
)

Mounted at /content/drive


## Install all dependencies

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 17.4 MB 623 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu113
    Uninstalling torch-1.12.0+cu113:
      Successfully uninstalled torch-1.12.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu113
    Uninstalling torchvision-0.13.0+cu113:
      Successfully uninstalled torchvision-0.13.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.8.1 which is incompatible.
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating ob

## Convert images to the required file format

In [ ]:
!mkdir /content/drive/MyDrive/formatted-pants-128-samples
!mkdir /content/drive/MyDrive/pants-generated-images/

In [ ]:
convert_script = "python /content/stylegan2-ada-pytorch/dataset_tool.py "\
  "--source /content/drive/MyDrive/resized-pants-128-samples "\
  "--dest /content/drive/MyDrive/formatted-pants-128-samples"

!{convert_script}

100% 128/128 [00:01<00:00, 123.79it/s]


In [ ]:
import glob
from PIL import Image

class ImageChecker:
  def __init__(self, folder_path, extension = ".png", colour_format = "RGB") -> None:
    self.extension = extension
    self.base_path = folder_path
    self.colour_format = colour_format
    self.files = glob.glob(self.base_path + "/*" + self.extension)
  
  def check_colour_format(self) -> bool:
    for f in self.files:
      img = Image.open(f)
      if img.mode != self.colour_format:
        return False
      img = None
    return True

  def check_size(self) -> bool:
    size_list = []
    for f in self.files:
      img = Image.open(f)
      size_list.append(img.size)
    return len(size_list) != 0 and len(set(size_list)) == 1 
    
  def get_files(self) -> list: 
    return self.files

In [ ]:
checker = ImageChecker(folder_path="/content/drive/MyDrive/formatted-pants-128-samples/00000")
print(len(checker.get_files()))
print(checker.check_colour_format())
print(checker.check_size())

128
True
True


## Train the GAN

In [ ]:
!pip install -U tb-nightly
!pip install -U future

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%tensorflow_version 1.x

In [ ]:
from tqdm.notebook import tqdm

train_script = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py "\
  f"--snap 2 --outdir /content/drive/MyDrive/pants-generated-images/ --data /content/drive/MyDrive/formatted-pants-128-samples/"

!{train_script}



Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 2,
  "network_snapshot_ticks": 2,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/formatted-pants-128-samples/",
    "use_labels": false,
    "max_size": 128,
    "xflip": false,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 1638

In [ ]:
!mkdir /content/drive/MyDrive/output-pants
s = "/usr/bin/python3 /content/stylegan2-ada-pytorch/generate.py --outdir=/content/drive/MyDrive/output-pants --trunc=1 --seeds=85,265,297,849 \
    --network=/content/drive/MyDrive/pants-generated-images/00001--auto1/network-snapshot-000192.pkl"

!{s}

mkdir: cannot create directory ‘/content/drive/MyDrive/output-pants’: File exists
Loading networks from "/content/drive/MyDrive/pants-generated-images/00001--auto1/network-snapshot-000192.pkl"...
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/generate.py", line 127, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/decorators.py", line 21, in new_f